<a href="https://colab.research.google.com/github/limpa105/RegexPlus/blob/main/ModelDevelopment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
train = pd.read_csv("const.tsv", sep='\t')

In [3]:
train = pd.read_csv("const.tsv", sep='\t')
train = train.groupby(["problem_id"]).first()
train = train.drop(['description', 'neg_examples'], axis=1)
train["const_values"] = [ len(i.split(","))-1!=0 for i in train["const_values"]]
train["var"] = [ np.asarray([ len(j) for j in i.split(" ")]).std() for i in train["pos_examples"]]
train["mean"] = [ np.asarray([ len(j) for j in i.split(" ")]).mean() for i in train["pos_examples"]]
train["optional"] = [("optional") in i for i in train["regex"]]
# how to get number of constants in common? interesection is too simple 
const = []
for i in range(len(train["pos_examples"])):
  l = train["pos_examples"][i].split(" ") 
  ans = l[0]
  for j in l:
    ans = set(j).intersection(ans)
  const.append(len(ans))
train["shared"] = const




In [ ]:
train["pos_examples"][0]

'5hhhpvRTIWJFHFJ 2hhhpfTHTJVJDIHXJIJ 4hhhCAGFJ 4hhhCAGJJ 4hhhrnSKOJJ 4hhhrnSKOCJ'

In [46]:
# predicting constants 
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(train[["var", "shared"]], train["const_values"])
np.unique(train["const_values"].values == clf.predict(train[["var", "shared"]]), return_counts= True)
print("Percent accuracy on train:", 551/(551+189))

Percent accuracy on train: 0.7445945945945946


In [ ]:
clf.predict

In [34]:
pickle.dump(clf, open("constants_model.sav", 'wb'))

In [ ]:
clf.predict(train[["var", "shared"]])[1]

True

In [4]:
# predicting optionals 
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(train[["var", "shared", "mean"]], train["optional"])
np.unique(train["optional"].values == clf.predict(train[["var", "shared", "mean"]]), return_counts= True)
print("Percent accuracy on train:", 613/(127+613))

Percent accuracy on train: 0.8283783783783784


In [5]:
pickle.dump(clf, open("optionals_model.sav", 'wb'))